In [ ]:
using MIDI, CSV, DataFrames

In [ ]:
function find_midi_files(directory::String)
    # Get a list of all the files and directories in the directory
    items = readdir(directory)

    # Initialize an empty vector to store the MIDI files
    midi_files = Vector{String}()

    # Loop through each item in the directory
    for item in items
        # Get the full path to the item
        item_path = joinpath(directory, item)

        # If the item is a directory, recursively search it for MIDI files
        if isdir(item_path)
            midi_files = vcat(midi_files, find_midi_files(item_path))
        # If the item is a MIDI file, add it to the list
        elseif endswith(item, ".mid") || endswith(item, ".midi")
            push!(midi_files, item_path)
        end
    end

    # Return the list of MIDI files
    return midi_files
end

In [ ]:
function parse_midi_files(midi_dir::String, output_dir::String)
    # Get a list of all the MIDI files in the directory
    midi_files = find_midi_files(midi_dir)
    # Loop through the MIDI files and parse them
    for midi_file in midi_files
        # Parse the MIDI file
        midi_data = load(midi_file)

        # Initialize empty vectors to store the parsed data
        track = midi_data.tracks[2]

        notes = getnotes(track)
        noteNames = []
        velocities = []
        times = []

        # Loop through the MIDI events and extract the relevant data
        for note in notes
            push!(noteNames, note.pitch)
            push!(velocities, note.velocity)
            push!(times, note.duration)
        end
        
        # Construct a DataFrame from the parsed data
        df = DataFrame(
            note=noteNames,
            velocity=velocities,
            time=times
        )

        # Write the DataFrame to a CSV file
        csv_file = joinpath(output_dir, replace(basename(midi_file), ".mid" => ".csv"))
        CSV.write(csv_file, df)
    end
end


In [ ]:
@time parse_midi_files("midiData", "csvData")

In [ ]:
function reconstruct_midi_file(csv_file::String, midi_file::String)
    # Read the CSV file into a DataFrame
    df = CSV.read(csv_file, DataFrame)

    # Convert the parsed data into Note objects
    notes = Notes()
    time = 0
    for i in 1:size(df, 1)
        # Calculate the time since the last MIDI event
        delta_time = round(Int, df.time[i])

        # Create a Note object for the note-on event
        note = Note(df.note[i], df.velocity[i], time, delta_time)
        push!(notes, note)

        # Update the time counter
        time += delta_time
    end

    # Create a MIDI track from the Note objects
    track = MIDITrack()
    addnotes!(track, notes)
    addtrackname!(track, "reconstructed track")

    # Create a MIDI file from the track
    file = MIDIFile()
    push!(file.tracks, track)
    writeMIDIFile(midi_file, file)
end


In [ ]:
reconstruct_midi_file("csvData/alb_esp1.csv", "reconstructed.mid")

In [163]:
function add_synthetic_anomalies(csv_file::String, anomaly_percentage::Float64)
    # Load the CSV file into a DataFrame
    df = CSV.read(csv_file, DataFrame)
    
    # Add a column to track anomalies
    df.anomalies = zeros(Int64, size(df, 1))

    selected_indices = Int[]
    for i in 1:num_anomalies
        # Choose a random index that hasn't been selected before
        index = rand(setdiff(1:size(df, 1), selected_indices))
        push!(selected_indices, index)

        # Create a new row with the values from the chosen index
        row = (note = df[index, :note], velocity = df[index, :velocity], time = df[index, :time])

        # Set one of the values to a semi-realistic anomalous value
        column_index = rand(1:size(df, 2))
        noteDeviation = rand(-14:14)
        velocityDeviation = rand(vcat(-40:-20, 20:40)) 
        timeDeviation = rand(vcat(-500:-50, 50:500))

        randDeviation = rand()

        if randDeviation < 0.33
            row = (note = row.note + noteDeviation, velocity = row.velocity, time = row.time, anomalies = 1)
        elseif randDeviation < 0.66
            row = (note = row.note, velocity = row.velocity + velocityDeviation, time = row.time, anomalies = 1)
        elseif randDeviation < 1
            row = (note = row.note, velocity = row.velocity, time = row.time + timeDeviation, anomalies = 1)
        end

        # Replace the original row with the new row containing the anomaly
        df[index, :] = row
    end



    # Write the DataFrame to the anomalous directory
    csv_file_name = splitdir(csv_file)[2]
    csv_output_file = joinpath("anomalous", csv_file_name)
    CSV.write(csv_output_file, df)
end


add_synthetic_anomalies (generic function with 3 methods)

In [164]:
csv_files = readdir("csvData")
for csv_file in csv_files
    add_synthetic_anomalies(joinpath("csvData", csv_file), 1.00)
end
